In [1]:
################################################################################
# MASTERARBEIT - SKRIPT 12:
# EXPERIMENT 3.4 (Forschungsfrage 3) - TABPFN REPLIKATION (LLM-Basis)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Forschungsfrage 3):
#
# 1. (Basis): Misst den Einfluss der vorherigen LLM-Inkonsistenzbehebung (FF2.2) 
#    auf die Ausreißererkennung unter Verwendung eines modernen KI-Modells.
#
# 2. (Datenbasis): Lädt den vom LLM reparierten Datensatz (2.2_rfd_repaired_llm.csv).
#
# 3. (Zielspalten): Erweitert die Analyse ('replies', 'views', 'votes') 
#    um die nun numerisch gewordenen Spalten ('price', 'saving').
#
# 4. (Voraussetzung): Führt die notwendige Zwangskonvertierung (Type Casting) 
#    durch, um die Daten in das PyTorch-Tensor-Format zu bringen.
#
# 5. (Speichern): Speichert die normalisierten Ausreißer-Werte (Scores) für die 
#    abschließende Evaluation (Skript 13).
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import os
import sys 
from sklearn.preprocessing import MinMaxScaler
import torch 
import time

# Importieren von TabPFN und den speziellen Erweiterungen
print("Lade notwendige Bibliotheken (Pandas, Numpy, PyTorch, TabPFN)...")
try:
    from tabpfn import TabPFNClassifier, TabPFNRegressor
    from tabpfn_extensions import unsupervised
    from tabpfn_extensions.unsupervised import experiments as unueberwachte_experimente
    print("TabPFN- und tabpfn-extensions-Bibliotheken wurden erfolgreich geladen.")
except ImportError as e:
    print("FEHLER: Kritische Bibliotheken (TabPFN/Torch) konnten nicht geladen werden.")
    sys.exit("Skript gestoppt, da Abhängigkeiten fehlen.")

# --- GLOBALE KONSTANTEN FÜR DIESES EXPERIMENT ---
DATEIPFAD_LLM_REPAIRED = 'ergebnisse/2.2_rfd_repaired_llm.csv'
DATEIPFAD_OUTPUT = 'ergebnisse/3.4_tabpfn_ausreisser_scores.csv'

# Zielspalten: FF1-Spalten + die durch FF2 reparierten Spalten
zielspalten = ['replies', 'views', 'votes', 'price', 'saving']

print("Alle Bibliotheken sind bereit.")
print("=" * 70)

#
################################################################################

# SCHRITT 2: Laden der LLM-reparierten Daten (FF2.2 Ergebnis)
print("--- Schritt 2: LLM-reparierte Daten laden (FF3 Basis) ---")

try:
    # LLM-Daten laden (Index wird mitgeladen)
    df_llm = pd.read_csv(DATEIPFAD_LLM_REPAIRED, index_col='original_index')
    df_llm = df_llm.reset_index(drop=True)
    print(f"Datensatz geladen: {DATEIPFAD_LLM_REPAIRED} ({df_llm.shape[0]} Zeilen)")
except FileNotFoundError:
    print(f"FEHLER: LLM-Ergebnisdatei '{DATEIPFAD_LLM_REPAIRED}' nicht gefunden.")
    sys.exit("Skript gestoppt.")

#
################################################################################

# --- SCHRITT 3: Datentyp-Konvertierung (Voraussetzung für TabPFN) ---
print("\n--- Schritt 3: Typ-Konvertierung und NaN-Handhabung ---")

# Methodische Notwendigkeit: TabPFN benötigt explizit Float-Daten.

# 1. Numerische Konvertierung erzwingen
try:
    print("STATUS: Erzwinge Konvertierung zu Float...")
    numerische_spalten_repariert = ['price', 'saving']
    
    for col in numerische_spalten_repariert:
        # Konvertierung zu Float erzwingen (behebt die Pandas-Object-Lesefehler)
        df_llm[col] = pd.to_numeric(df_llm[col], errors='coerce')
    
    # 2. NaN-Handhabung: TabPFN kann keine NaN verarbeiten. Wir füllen NaNs mit 0.
    # Dies füllt alle NaN-Werte (auch die nicht reparierten leeren Zellen) mit 0.
    df_llm[zielspalten] = df_llm[zielspalten].fillna(0)
    
    print("STATUS: Alle Zielspalten erfolgreich in Float konvertiert und NaN mit 0 gefüllt.")
    print(df_llm[zielspalten].dtypes)
    
except Exception as e:
    print(f"FEHLER bei der Typkonvertierung: {e}")
    sys.exit("Skript gestoppt.")

#
################################################################################

# SCHRITT 4: Datenkonvertierung zu PyTorch-Tensoren
print("\n--- Schritt 4: Datenkonvertierung zu PyTorch-Tensoren ---")

# TabPFN basiert auf PyTorch. Die Daten müssen in das 'Tensor'-Format konvertiert werden.

# 1. Extrahieren der Zieldaten (5 Dimensionen)
X_data = df_llm[zielspalten]

# 2. Konvertierung der Daten (X) in einen Float-Tensor
try:
    X_tensor = torch.tensor(X_data.values).float()
    print(f"Daten (X) erfolgreich in Tensor konvertiert (Shape: {X_tensor.shape})")
except Exception as e:
    print(f"FEHLER bei der Konvertierung zu Tensor: {e}")
    sys.exit("Skript gestoppt.")

# 3. Definieren der Zielvariable (y) und Attributnamen
y_tensor = None
attribute_namen = zielspalten

print("=" * 70)

#
################################################################################

# SCHRITT 5: Initialisierung des unüberwachten TabPFN-Modells
print("\n--- Schritt 5: Initialisierung des unüberwachten TabPFN-Modells ---")

# Technische Parameter (FF1-Replikation):
# n_estimators = 32 (Kompromiss für Laufzeit)
# ignore_pretraining_limits = True (wegen 1326 Zeilen > 1024 Limit)

try:
    print("Initialisiere TabPFNClassifier und TabPFNRegressor...")
    clf = TabPFNClassifier(n_estimators=32, ignore_pretraining_limits=True)
    reg = TabPFNRegressor(n_estimators=32, ignore_pretraining_limits=True)
    
    model_unueberwacht = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf=clf, tabpfn_reg=reg
    )
    print("Das unüberwachte TabPFN-Modell wurde erfolgreich initialisiert.")
except Exception as e:
    print(f"FEHLER bei der Initialisierung des Modells: {e}")
    sys.exit("Skript gestoppt.")
print("=" * 70)

#
################################################################################

# SCHRITT 6: Ausführung des TabPFN-Ausreißer-Experiments
print("\n--- Schritt 6: Ausführung des TabPFN-Ausreißer-Experiments ---")

# 1. Initialisierung der Experiment-Klasse
try:
    exp_ausreisser = unueberwachte_experimente.OutlierDetectionUnsupervisedExperiment(
        task_type="unsupervised"
    )
    print("Das Experiment (OutlierDetection) wurde initialisiert.")
except Exception as e:
    print(f"FEHLER bei der Initialisierung des Experiments: {e}")
    sys.exit("Skript gestoppt.")

print(f"Starte die Ausführung (.run) auf den Daten (Shape: {X_tensor.shape})...")
print("Dieser Schritt ist rechenintensiv und kann einige Minuten dauern.")

# 2. Ausführung des Experiments (.run) MIT LAUFZEITMESSUNG
try:
    # Laufzeitmessung START (Experiment 3.4, TabPFN auf LLM-Basis)
    start_zeit = time.time()

    ergebnisse = exp_ausreisser.run(
        tabpfn=model_unueberwacht,
        X=X_tensor,
        y=y_tensor,
        attribute_names=attribute_namen,
        should_plot=False
    )

    # Laufzeitmessung ENDE
    end_zeit = time.time()
    laufzeit_sek = end_zeit - start_zeit

    print("Das Experiment wurde erfolgreich abgeschlossen.")
    print(f"TabPFN-Laufzeit (Experiment 3.4, LLM-Basis): {laufzeit_sek:.2f} Sekunden "
          f"(≈ {laufzeit_sek/60:.2f} Minuten).")
    print(f"Durchschnittliche Laufzeit pro Zeile: {laufzeit_sek / X_tensor.shape[0]:.4f} Sekunden.")

except Exception as e:
    print(f"FEHLER während der Ausführung des Experiments (.run): {e}")
    sys.exit("Skript gestoppt.")
#
################################################################################

# SCHRITT 7: Ergebnisse extrahieren und verarbeiten
print("\n--- Schritt 7: Ergebnisse extrahieren und verarbeiten ---")

# TabPFN gibt 'outlier_scores' (Werte) zurück, die wir normalisieren müssen.
ausreisser_werte = ergebnisse.get('outlier_scores')

if ausreisser_werte is None:
    print("FEHLER: Schlüssel 'outlier_scores' wurde in den Ergebnissen nicht gefunden.")
    sys.exit("Skript gestoppt.")

# Die 'ausreisser_werte' werden in den ursprünglichen DataFrame integriert.
df_ergebnisse_tabpfn = df_llm.copy()
df_ergebnisse_tabpfn['tabpfn_ausreisser_wert'] = ausreisser_werte

# Methodische Notwendigkeit: Normalisierung auf 0 bis 1, um die Scores interpretierbar zu machen.
roh_werte = df_ergebnisse_tabpfn[['tabpfn_ausreisser_wert']].values
skalierer = MinMaxScaler()
df_ergebnisse_tabpfn['tabpfn_ausreisser_wert_norm'] = skalierer.fit_transform(roh_werte)

print("Die Ausreißer-Werte wurden auf 0-1 normalisiert ('tabpfn_ausreisser_wert_norm').")

# Ausgabe der 10 höchsten normalisierten Ausreißer-Werte als Kontrolle
print("\nDie 10 Zeilen mit den höchsten (normalisierten) Ausreißer-Werten:")
print(
    df_ergebnisse_tabpfn
    .sort_values(by='tabpfn_ausreisser_wert_norm', ascending=False)
    [zielspalten + ['tabpfn_ausreisser_wert_norm']]
    .head(10)
    .round(4)
)
print("=" * 70)

#
################################################################################

# SCHRITT 8: Ergebnisse speichern und Zusammenfassung
print("\n--- Schritt 8: Ergebnisse speichern und Zusammenfassung ---")
print("Dieses Skript (12) ist nun abgeschlossen.")

# Sicherstellen, dass der Speicher-Ordner ('ergebnisse') existiert
os.makedirs('ergebnisse', exist_ok=True)

# 1. Definieren des Dateipfads für die Ergebnisse
ergebnis_dateipfad = DATEIPFAD_OUTPUT

# 2. Speichern des DataFrames (Scores werden gespeichert)
df_ergebnisse_tabpfn.to_csv(
    ergebnis_dateipfad,
    columns=['tabpfn_ausreisser_wert', 'tabpfn_ausreisser_wert_norm'],
    index=True
)

print(f"\nErgebnisse (Werte/Scores) wurden in '{ergebnis_dateipfad}' gespeichert.")

# --- ZUSAMMENFASSUNG (Für das Protokoll/die Masterarbeit) ---
print("\n--- ZUSAMMENFASSUNG EXPERIMENT 3.4 (TabPFN Replikation - LLM-Basis) ---")
print("Methode:           Moderne KI: TabPFN (unüberwacht)")
print(f"Basisdaten:        {DATEIPFAD_LLM_REPAIRED}")
print(f"Zielspalten:       {zielspalten} (als 5D-Merkmalsraum)")
print(f"Parameter (n_est): {clf.n_estimators}")
print(f"Parameter (ignore):{clf.ignore_pretraining_limits}")
print("ERGEBNIS (Art):    Ausreißer-Werte (Scores) für alle 1326 Zeilen wurden erzeugt und gespeichert.")
print("=" * 70)


Lade notwendige Bibliotheken (Pandas, Numpy, PyTorch, TabPFN)...
TabPFN- und tabpfn-extensions-Bibliotheken wurden erfolgreich geladen.
Alle Bibliotheken sind bereit.
--- Schritt 2: LLM-reparierte Daten laden (FF3 Basis) ---
Datensatz geladen: ergebnisse/2.2_rfd_repaired_llm.csv (1326 Zeilen)

--- Schritt 3: Typ-Konvertierung und NaN-Handhabung ---
STATUS: Erzwinge Konvertierung zu Float...
STATUS: Alle Zielspalten erfolgreich in Float konvertiert und NaN mit 0 gefüllt.
replies      int64
views        int64
votes        int64
price      float64
saving     float64
dtype: object

--- Schritt 4: Datenkonvertierung zu PyTorch-Tensoren ---
Daten (X) erfolgreich in Tensor konvertiert (Shape: torch.Size([1326, 5]))

--- Schritt 5: Initialisierung des unüberwachten TabPFN-Modells ---
Initialisiere TabPFNClassifier und TabPFNRegressor...
Das unüberwachte TabPFN-Modell wurde erfolgreich initialisiert.

--- Schritt 6: Ausführung des TabPFN-Ausreißer-Experiments ---
Das Experiment (OutlierDetectio